# 1 Page Rank 
## a)


In [2]:
import numpy as np
m_a = np.asarray([[0 , 0.5, 0.5],[0, 0, 1],[0, 0, 0]]).T
display("Matrix A:",m_a)

m_b = np.asarray([[0 , 0.5, 0.5],[0, 0, 1],[0, 0, 1]]).T

display("Matrix B:",m_b)

'Matrix A:'

array([[0. , 0. , 0. ],
       [0.5, 0. , 0. ],
       [0.5, 1. , 0. ]])

'Matrix B:'

array([[0. , 0. , 0. ],
       [0.5, 0. , 0. ],
       [0.5, 1. , 1. ]])

In [3]:
def page_rank_naive(matrix, max_step = 10):
    n = len(matrix)
    state = np.asarray([1 / n] * n)
    
    rslt = []
    rslt.append(state)
    for i in range(max_step):
        state = matrix.dot(state)
        rslt.append(state)
    
    return rslt

In [4]:
s = page_rank_naive(m_a)
print("Matrix a")
for i,step in enumerate(s):
    print(i,step)

s = page_rank_naive(m_b)
print("Matrix b")
for i,step in enumerate(s):
    print(i,step)

Matrix a
0 [0.33333333 0.33333333 0.33333333]
1 [0.         0.16666667 0.5       ]
2 [0.         0.         0.16666667]
3 [0. 0. 0.]
4 [0. 0. 0.]
5 [0. 0. 0.]
6 [0. 0. 0.]
7 [0. 0. 0.]
8 [0. 0. 0.]
9 [0. 0. 0.]
10 [0. 0. 0.]
Matrix b
0 [0.33333333 0.33333333 0.33333333]
1 [0.         0.16666667 0.83333333]
2 [0. 0. 1.]
3 [0. 0. 1.]
4 [0. 0. 1.]
5 [0. 0. 1.]
6 [0. 0. 1.]
7 [0. 0. 1.]
8 [0. 0. 1.]
9 [0. 0. 1.]
10 [0. 0. 1.]


-> A leads to all probabilities becoming zero as we have a dead end  
-> B leads to all probability accumulating in the spider trap loop

In [5]:
def has_dead_end(matrix, return_i = False):
    for i in range(len(matrix)):
        if matrix[:,i].sum() == 0:
            if return_i:
                return True, i
            return True
    return False


In [6]:
print(has_dead_end(m_b))

False


In [20]:
def page_rank(matrix, max_step = 10, use_beta = True, dead_ends = True, all_results = True):
    beta = 0.85
    n = len(matrix)
    state = np.asarray([1 / n] * n)
    old = state.copy()
    old_matrix = matrix.copy()
    deleted = []
    if dead_ends:
        try:
            while has_dead_end(matrix):
                _, i = has_dead_end(matrix,True)
                print(i)
                deleted.append(i)
                idx = list(set(range(matrix.shape[0])).difference([i]))
                matrix = matrix[np.ix_(idx,idx)]
                sums = matrix.sum(axis=0,keepdims=1); 
                sums[sums==0] = 1
                matrix = matrix/sums
                if matrix.shape == (0,0):
                    raise Exception("Too many Deadends")
        except:
            print("No nodes left when deleting deadends. Aborting..")
            return []
    rslt = [state]

    for i in range(max_step):
        if use_beta:
            state = matrix.dot(state) + (1-beta) * old
        else:
            state = matrix.dot(state)
        rslt.append(state)
        
    for i in deleted[::-1]:
        #TODO: Implement adding back the stuff
        ranks = rslt[max_step]
        pre_decessors = np.where(old_matrix[:,i] > 0)
        pre_d_power = [1 / len(old_matrix[t].nonzero()) for t in pre_decessors]
        rank = np.sum([s * t for s,t in zip(pre_d_power, ranks[pre_decessors])])
        rslt.append(rslt[:-1].append(rank))
        
    if all_results:
        return rslt
    else:
        return rslt.pop()

In [21]:
s = np.random.random(1)[0]
s

0.567123152437463

In [22]:
print("DeadEnd")
s = page_rank(m_a, use_beta = False)
print("Matrix a")
for i,step in enumerate(s):
    print(i,step)

s = page_rank(m_b, use_beta = False)
print("Matrix b")
for i,step in enumerate(s):
    print(i,step)
    
    
print("Taxation")
s = page_rank(m_a, dead_ends = False)
print("Matrix a")
for i,step in enumerate(s):
    print(i,step)

s = page_rank(m_b, dead_ends = False)
print("Matrix b")
for i,step in enumerate(s):
    print(i,step)
    
print("Both")

s = page_rank(m_a)
print("Matrix a")
for i,step in enumerate(s):
    print(i,step)

s = page_rank(m_b)
print("Matrix b")
for i,step in enumerate(s):
    print(i,step)

DeadEnd
2
1
0
No nodes left when deleting deadends. Aborting..
Matrix a
Matrix b
0 [0.33333333 0.33333333 0.33333333]
1 [0.         0.16666667 0.83333333]
2 [0. 0. 1.]
3 [0. 0. 1.]
4 [0. 0. 1.]
5 [0. 0. 1.]
6 [0. 0. 1.]
7 [0. 0. 1.]
8 [0. 0. 1.]
9 [0. 0. 1.]
10 [0. 0. 1.]
Taxation
Matrix a
0 [0.33333333 0.33333333 0.33333333]
1 [0.05       0.21666667 0.55      ]
2 [0.05       0.075      0.29166667]
3 [0.05  0.075 0.15 ]
4 [0.05  0.075 0.15 ]
5 [0.05  0.075 0.15 ]
6 [0.05  0.075 0.15 ]
7 [0.05  0.075 0.15 ]
8 [0.05  0.075 0.15 ]
9 [0.05  0.075 0.15 ]
10 [0.05  0.075 0.15 ]
Matrix b
0 [0.33333333 0.33333333 0.33333333]
1 [0.05       0.21666667 0.88333333]
2 [0.05  0.075 1.175]
3 [0.05  0.075 1.325]
4 [0.05  0.075 1.475]
5 [0.05  0.075 1.625]
6 [0.05  0.075 1.775]
7 [0.05  0.075 1.925]
8 [0.05  0.075 2.075]
9 [0.05  0.075 2.225]
10 [0.05  0.075 2.375]
Both
2
1
0
No nodes left when deleting deadends. Aborting..
Matrix a
Matrix b
0 [0.33333333 0.33333333 0.33333333]
1 [0.05       0.21666667

Deadends  
-> A: As there is no node left when removing deadends, we are left without a page rank
-> B: As we do not have dead ends, no change  
Taxation  
-> A: As the surfer disapears with change beta, sum is less than 1
-> B: Works well, spider trap does not accumulate all of the rank anymore
Both
-> A: See above for deadends, same issue occurs
-> B: See above for taxation, there is no change as we do not have deadends

# 2
## a

In [94]:
import pandas as pd
data = pd.read_csv("material/stack_network_links.csv")
transition_matrix = pd.crosstab(data["source"], data["target"]).T
transition_matrix = transition_matrix.div(transition_matrix.sum(axis=1), axis=0)
display(transition_matrix)
display(transition_matrix.values.sum(axis = 1))

source,.net,agile,ajax,amazon-web-services,android,android-studio,angular,angular2,angularjs,apache,...,visual-studio,vue.js,wcf,web-services,windows,wordpress,wpf,xamarin,xcode,xml
target,,,,,,,,,,,,,,,,,,,,,
.net,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.125000,0.0,0.0,0.0,0.125,0.0,0.0,0.0
agile,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000,0.0,0.0,0.0
ajax,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000,0.0,0.0,0.0
amazon-web-services,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000,0.0,0.0,0.0
android,0.000000,0.0,0.0,0.0,0.0,0.333333,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
wordpress,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000,0.0,0.0,0.0
wpf,0.166667,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.166667,0.0,0.0,0.0,0.000,0.0,0.0,0.0
xamarin,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000,0.0,0.0,0.0


array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [93]:
s = page_rank(transition_matrix.values, all_results=False)
rslt = pd.DataFrame(s, index = transition_matrix.index)
rslt.sum()

0    1.0
dtype: float64

# 4

In [7]:
import sys

from itertools import chain, combinations
from collections import defaultdict
from optparse import OptionParser


def subsets(arr):
    """ Returns non empty subsets of arr"""
    return chain(*[combinations(arr, i + 1) for i, a in enumerate(arr)])


def returnItemsWithMinSupport(itemSet, transactionList, minSupport, freqSet):
    """calculates the support for items in the itemSet and returns a subset
    of the itemSet each of whose elements satisfies the minimum support"""
    _itemSet = set()
    localSet = defaultdict(int)

    for item in itemSet:
        for transaction in transactionList:
            if item.issubset(transaction):
                freqSet[item] += 1
                localSet[item] += 1

    for item, count in localSet.items():
        support = float(count) / len(transactionList)

        if support >= minSupport:
            _itemSet.add(item)

    return _itemSet

def joinSet(itemSet, length):
    """Join a set with itself and returns the n-element itemsets"""
    return set(
        [i.union(j) for i in itemSet for j in itemSet if len(i.union(j)) == length]
    )


def getItemSetTransactionList(data_iterator):
    transactionList = list()
    itemSet = set()
    for record in data_iterator:
        transaction = frozenset(record)
        transactionList.append(transaction)
        for item in transaction:
            itemSet.add(frozenset([item]))  # Generate 1-itemSets
    return itemSet, transactionList


def runApriori(data_iter, minSupport, minConfidence):
    """
    run the apriori algorithm. data_iter is a record iterator
    Return both:
     - items (tuple, support)
     - rules ((pretuple, posttuple), confidence)
    """
    itemSet, transactionList = getItemSetTransactionList(data_iter)
    print(itemSet)
    print(transactionList)
    freqSet = defaultdict(int)
    largeSet = dict()
    # Global dictionary which stores (key=n-itemSets,value=support)
    # which satisfy minSupport

    assocRules = dict()
    # Dictionary which stores Association Rules

    oneCSet = returnItemsWithMinSupport(itemSet, transactionList, minSupport, freqSet)

    currentLSet = oneCSet
    k = 2
    while currentLSet != set([]):
        largeSet[k - 1] = currentLSet
        currentLSet = joinSet(currentLSet, k)
        currentCSet = returnItemsWithMinSupport(
            currentLSet, transactionList, minSupport, freqSet
        )
        currentLSet = currentCSet
        k = k + 1

    def getSupport(item):
        """local function which Returns the support of an item"""
        return float(freqSet[item]) / len(transactionList)

    toRetItems = []
    for key, value in largeSet.items():
        toRetItems.extend([(tuple(item), getSupport(item)) for item in value])

    toRetRules = []
    for key, value in list(largeSet.items())[1:]:
        for item in value:
            _subsets = map(frozenset, [x for x in subsets(item)])
            for element in _subsets:
                remain = item.difference(element)
                if len(remain) > 0:
                    confidence = getSupport(item) / getSupport(element)
                    if confidence >= minConfidence:
                        toRetRules.append(((tuple(element), tuple(remain)), confidence))
    return toRetItems, toRetRules


def printResults(items, rules):
    """prints the generated itemsets sorted by support and the confidence rules sorted by confidence"""
    for item, support in sorted(items, key=lambda x: x[1]):
        print("item: %s , %.3f" % (str(item), support))
    print("\n------------------------ RULES:")
    for rule, confidence in sorted(rules, key=lambda x: x[1]):
        pre, post = rule
        print("Rule: %s ==> %s , %.3f" % (str(pre), str(post), confidence))


def to_str_results(items, rules):
    """prints the generated itemsets sorted by support and the confidence rules sorted by confidence"""
    i, r = [], []
    for item, support in sorted(items, key=lambda x: x[1]):
        x = "Support item: %s , %.3f" % (str(item), support)
        i.append(x)

    for rule, confidence in sorted(rules, key=lambda x: x[1]):
        pre, post = rule
        x = "Confidence Rule: %s ==> %s , %.3f" % (str(pre), str(post), confidence)
        r.append(x)

    return i, r





In [8]:
transactions = [
    ["I1", "I2", "I5"],
    ["I2", "I4"],
    ["I2", "I3"],
    ["I1", "I2", "I4"],
    ["I1", "I3"],
    ["I2", "I3"],
    ["I1", "I3"],
    ["I1", "I2", "I3", "I5"],
    ["I1", "I2", "I3"],
]
items, rules = runApriori(transactions, 0.15, 0.6)

printResults(items, rules)

{frozenset({'I3'}), frozenset({'I1'}), frozenset({'I2'}), frozenset({'I5'}), frozenset({'I4'})}
[frozenset({'I1', 'I2', 'I5'}), frozenset({'I4', 'I2'}), frozenset({'I3', 'I2'}), frozenset({'I1', 'I2', 'I4'}), frozenset({'I1', 'I3'}), frozenset({'I3', 'I2'}), frozenset({'I1', 'I3'}), frozenset({'I1', 'I3', 'I2', 'I5'}), frozenset({'I1', 'I3', 'I2'})]
item: ('I5',) , 0.222
item: ('I4',) , 0.222
item: ('I1', 'I5') , 0.222
item: ('I2', 'I5') , 0.222
item: ('I4', 'I2') , 0.222
item: ('I1', 'I2', 'I5') , 0.222
item: ('I1', 'I3', 'I2') , 0.222
item: ('I1', 'I2') , 0.444
item: ('I3', 'I2') , 0.444
item: ('I1', 'I3') , 0.444
item: ('I3',) , 0.667
item: ('I1',) , 0.667
item: ('I2',) , 0.778

------------------------ RULES:
Rule: ('I1',) ==> ('I2',) , 0.667
Rule: ('I3',) ==> ('I2',) , 0.667
Rule: ('I1',) ==> ('I3',) , 0.667
Rule: ('I3',) ==> ('I1',) , 0.667
Rule: ('I5',) ==> ('I1',) , 1.000
Rule: ('I5',) ==> ('I2',) , 1.000
Rule: ('I4',) ==> ('I2',) , 1.000
Rule: ('I5',) ==> ('I1', 'I2') , 1.000
